职责链模式

使多个对象都有机会处理请求，从而避免请求的发送者和接收者之间的耦合关系，将这些对象连成一条链，并沿着这条链传递该请求，直到有一个对象处理它为止。

In [1]:
(function(){
    var order = function(orderType, pay, stock){
        if(orderType === 1){
            if(pay === true){
                console.log('500元定金预购')
            }else{
                if(stock > 0){
                    console.log('普通购买')
                }else{
                    console.log('手机库存不足')
                }
            }
        }else if(orderType === 2){
            if(pay === true){
                console.log('200元定金预购')
            }else{
                if(stock > 0){
                    console.log('普通购买')
                }else{
                    console.log('手机库存不足')
                }
            }
        }else if(orderType === 3){
            if(stock > 0){
                console.log('普通购买')
            }else{
                console.log('手机库存不足')
            }
        }
    }
    
    order(1,true,500)
})()

500元定金预购


In [4]:
(function(){
    var order500 = function(orderType,pay,stock){
        if(orderType===1&&pay===true){
            console.log('200 -> 100')
        }else{
            order200(orderType,pay,stock);
        }
    }
    var order200 = function(orderType,pay,stock){
        if(orderType===2&&pay===true){
            console.log('100 -> 50')
        }else{
            orderNormal(orderType,pay,stock);
        }
    }
    var orderNormal = function(orderType,pay,stock){
        if(stock > 0){
            console.log('normal')
        }else{
            console.log('no stock')
        }
    }
    
    order500(1,true,500)
    order500(1,false,500)
})()

200 -> 100
normal


In [7]:
(function(){
    var order500 = function(orderType,pay,stock){
        if(orderType===1&&pay===true){
            console.log('200 -> 100')
        }else{
            return 'nextSuccessor'
        }
    }
    var order200 = function(orderType,pay,stock){
        if(orderType===2&&pay===true){
            console.log('100 -> 50')
        }else{
            return 'nextSuccessor'
        }
    }
    var orderNormal = function(orderType,pay,stock){
        if(stock > 0){
            console.log('normal')
        }else{
            console.log('no stock')
        }
    }
    
    var Chain = function(fn){
        this.fn = fn;
        this.successor = null
    }
    
    Chain.prototype.setNextSuccessor = function(successor){
        return this.successor = successor
    }
    
    Chain.prototype.passRequest = function(){
        var ret = this.fn.apply(this,arguments)
        if(ret === 'nextSuccessor'){
            return this.successor && this.successor.passRequest.apply(this.successor,arguments)
        }
        return ret;
    }
    
    var chainOrder500 = new Chain(order500);
    var chainOrder200 = new Chain(order200);
    var chainOrderNormal = new Chain(orderNormal);
    
    chainOrder500.setNextSuccessor(chainOrder200)
    chainOrder200.setNextSuccessor(chainOrderNormal)
    chainOrder500.passRequest(1,true,500)
    chainOrder500.passRequest(2,true,500)
})()

200 -> 100
100 -> 50


In [8]:
(function(){
    var Chain = function(fn){
        this.fn = fn;
        this.successor = null
    }
    
    Chain.prototype.setNextSuccessor = function(successor){
        return this.successor = successor
    }
    
    Chain.prototype.passRequest = function(){
        var ret = this.fn.apply(this,arguments)
        if(ret === 'nextSuccessor'){
            return this.successor && this.successor.passRequest.apply(this.successor,arguments)
        }
        return ret;
    }
    
    Chain.prototype.next = function(){
        return this.successor && this.successor.passRequest.apply(this.successor, arguments)
    }
    
    var fn1 = new Chain(function(){
        console.log(1);
        return 'nextSuccessor';
    })
    
    var fn2 = new Chain(function(){
        console.log(2);
        var self = this;
        setTimeout(function(){
            self.next()
        },1000)
    })
    
    var fn3 = new Chain(function(){
        console.log(3)
    })
    
    fn1.setNextSuccessor(fn2).setNextSuccessor(fn3);
    fn1.passRequest()
})()

1
2
3


In [9]:
(function(){
    Function.prototype.after = function(fn){
        var self = this;
        return function(){
            var ret = self.apply(this,arguments);
            if(ret === 'nextSuccessor'){
                return fn.apply(this,arguments)
            }
            
            return ret;
        }
    }
    
    var order = (function(){
        console.log(1);
        return 'nextSuccessor'
    }).after((function(){
        console.log(2);
        return 'nextSuccessor'
    })).after((function(){
        console.log(3);
        return 'over'
    }))
    
    order()
})()

2:24 - Property 'after' does not exist on type 'Function'.
17:8 - Property 'after' does not exist on type '() => string'.
